# Practice Session AI: Decision Tree <a class="tocSkip">

In [4]:
# vul in

print("Naam:", "Kennes")
print("Voornaam:", "Dries")
print("S-nummer:", "R0486630")
print("Richting:", "ICT")

# druk <ctrl> + <enter>

Naam: Kennes
Voornaam: Dries
S-nummer: R0486630
Richting: ICT


<!-- <img src="https://i.imgur.com/kTl5dQa.jpg" alt="panda in tree" width=500/> -->

In deze labosessie gaan we zelf één van de *machine learning models* uit de vorige notebook implementeren, namelijk de *Decision Tree*. Net zoals in de vorige sessies levert de *Pandas* library hier de basisstructuren waarin de data wordt voorgesteld. Om de werking van het uitgewerkte algoritme te testen, zullen we dit evalueren op de vertrouwde wijn-dataset.

## Imports and Loading

Om te veel herhaling te voorkomen, wordt de code voor deze stappen gewoon meegegeven. Net zoals vorige week gaan we werken met een ``ModelFrame`` uit Pandas_ML, omwille van de handige ``data`` (*features*) en ``target`` (*target*) accessoren.

In [5]:
import numpy as np
import pandas as pd
import pandas_ml as pdml

from tqdm import tqdm        ## process bar tool (optional)

In [6]:
col_names = ['type', 'alcohol', 'malic_acid', 'ash', 'alkalinity', 'magnesium', 'total_phenols', 'flavonoids', 'nonflavonoid_phenols', 'proanthocyanins', 'color_intensity', 'color_hue', 'OD280', 'proline']
df = pd.read_csv("data/wine_orig.csv", names=col_names)

In [7]:
df.head()

,type,alcohol,malic_acid,ash,alkalinity,magnesium,total_phenols,flavonoids,nonflavonoid_phenols,proanthocyanins,color_intensity,color_hue,OD280,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
df_wine = pdml.ModelFrame(df, target='type')

In [9]:
train_wine, test_wine = df_wine.model_selection.train_test_split(test_size=0.3, random_state=0)
print("Length of training set: {} ({}% of the samples)".format(len(train_wine), len(train_wine)/len(df_wine)))
print("Length of test set: {} ({}% of the samples)".format(len(test_wine), len(test_wine)/len(df_wine)))

Length of training set: 124 (0.6966292134831461% of the samples)
Length of test set: 54 (0.30337078651685395% of the samples)


In [10]:
train_wine.head()

,type,alcohol,malic_acid,ash,alkalinity,magnesium,total_phenols,flavonoids,nonflavonoid_phenols,proanthocyanins,color_intensity,color_hue,OD280,proline
22,1,13.71,1.86,2.36,16.6,101.0,2.61,2.88,0.27,1.69,3.80,1.11,4.00,1035.0
108,2,12.22,1.29,1.94,19.0,92.0,2.36,2.04,0.39,2.08,2.70,0.86,3.02,312.0
175,3,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
145,3,13.16,3.57,2.15,21.0,102.0,1.50,0.55,0.43,1.30,4.00,0.60,1.68,830.0
71,2,13.86,1.51,2.67,25.0,86.0,2.95,2.86,0.21,1.87,3.38,1.36,3.16,410.0


## Building Blocks

### Splitter

Eerst en vooral is er een functie nodig die een ``ModelFrame`` opsplitst in twee delen op basis van de waarde van één van de *features*, en deze twee delen teruggeeft.

In [ ]:
def split_on_attribute(df_node, attribute, value):
    # <, >=
    # print('split_on_attribute:', attribute, value)
    return df_node.loc[df_node[attribute] < value], df_node.loc[df_node[attribute] >= value]

#split_on_attribute(df_wine, 'alcohol', 13)

### Cost function

De split-functie gaat bij het opstellen van de tree opgeroepen worden wanneer er een *node* binair moet worden gesplitst. Uiteraard is het op voorhand nogal moeilijk te weten welke *feature* en bijbehorende *value* moeten gebruikt worden om tot een zo goed mogelijke split te komen waarbij de klassen zo volledig mogelijk van elkaar te scheiden. Om dit te testen maken we gebruik van enkele concepten uit de informatietheorie, waardoor we tot een *cost function* kunnen komen. 

#### Information Gain and Entropy

*Information gain* is een manier om uit te drukken hoeveel onzekerheid er wordt verloren bij het opsplitsen van de data in een *node*. Deze onzekerheid wordt uitgedrukt in de vorm van *entropy* (eenheid: *bits*), beschreven in de onderstaande formule.

\begin{equation*}
H(X) = - \sum \limits_{i=1}^{n} p(x_i) \log_2 p(x_i)
\end{equation*}

Om de *information gain* tussen een *node* en zijn *children* te maximaliseren, is het voldoende om de split te kiezen waarbij de *entropy* van de *children* minimaal is.

In [ ]:
def calc_px(df_node):
    return [df_node.loc[df['type'] == x].size/df_node.size for x in df_node['type'].unique()]

def entropy(df_node):
    px = calc_px(df_node)
    return -sum(px * np.log2(px))

In [13]:
# test entropy
#entropy(train_wine)

#### Gini Impurity

Een andere manier om de split te optimaliseren is de *Gini impurity*. Deze is een maat voor de kans dat elementen van een bepaalde klasse in een *node* gemisclassificeerd worden. 

\begin{equation*}
I_G(X) = 1 - \sum \limits_{i=1}^{n} p(x_i)^2
\end{equation*}

Uiteraard willen we dit aantal misclassificaties zo laag mogelijk houden, dus voor een zo goed mogelijke split hebben we graag een minimale *Gini impurity* bij de *children* van de *node*.

In [14]:
def gini_index(df_node):
    return 1 - sum(x*x for x in calc_px(df_node))

In [15]:
# test Gini impurity
#gini_index(train_wine)

#### Weighted Cost

Om de *costs* van de *children* van een *node* in één getal te kunnen weergeven, berekenen we het gewogen gemiddelde van de *splitting cost* voor zowel *entropy* als *Gini impurity* op de volgende manier:

\begin{equation*}
\overline{cost}(N, L, R) = P(L \vert N) \cdot cost(L) + P(R \vert N) \cdot cost(R)
\end{equation*}

In [16]:
def weighted_cost(df_node, node_left, node_right, cost_func):
    return node_left.size / df_node.size * cost_func(node_left) + \
           node_right.size / df_node.size * cost_func(node_right)

In [17]:
# test weighted cost with entropy
weighted_cost(train_wine, *split_on_attribute(train_wine, 'alkalinity', 16), entropy)

1.4577427356053494

In [18]:
# test weighted cost with Gini impurity
weighted_cost(train_wine, *split_on_attribute(train_wine, 'alkalinity', 16), gini_index)

0.6118637247669506

### Apply Split

Nu hoeven we enkel nog een functie te schrijven die over alle waarden van alle *features* in de dataset gaat, hierbij de *cost* berekent, en vervolgens *feature* en waarde teruggeeft waarbij de *splitting cost* het laagste is. 

**Let op:** Op de ``target`` kolom mag **nooit** gesplitst worden!

In [19]:
def find_best_split(df_node, cost_func):
    """
    :param df_node: 
    :param cost_func: 
    :return: (attribute: str, value:float, left, right)
    """
    best_costs = None
    best_col = None
    best_val = None
    for col in df_node.columns.values:
        if col == 'type':
            continue
        for val in df_node[col].unique():
            l, r = split_on_attribute(df_node, col, val)
            cost = weighted_cost(df_node, l, r, cost_func)
            if best_costs is None or cost < best_costs:
                best_col = col
                best_val = val
                best_l = l
                best_r = r
    return best_col, best_val, best_l, best_r

In [ ]:
# test best split finder with entropy
# find_best_split(train_wine, gini_index)

In [21]:
#test best split finder with Gini impurity
# find_best_split(train_wine, entropy)

### Build Tree

Met behulp van de ``find_best_split`` functie is het nu mogelijk om de *decision tree* recursief op te stellen. Het grootste deel van de functies en attributen van de ``TreeNode`` klasse zijn reeds gegeven, enkel de ``split`` functie (waar de recursie gebeurt) dient nog aangevuld te worden.

In [22]:
class TreeNode(object):
    """ 
    Forms the node of a decision tree.
    
    Args:
        level: Level of the node in the tree.
        df_node: ModelFrame containing the data of the node
        cost_func: Function to calculate the splitting cost (entropy or gini_index).
        max_depth: Maximum depth of the tree.
        min_length: Minimum amount of elements that a node has to contain to be considered splittable.
    """
    counter = 0
    def __init__(self, level, df_node, cost_func, max_depth, min_length, **kwargs):
        self._id = type(self).counter
        type(self).counter += 1
        self.level = level
        self.df_node = df_node
        self.cost_func = cost_func
        self.max_depth = max_depth
        self.min_length = min_length
        self.cost = self.cost_func(self.df_node)
        self.split_attr = None
        self.split_value = None
        self.left = None
        self.right = None
    
    @property
    def record_amt(self):
        """ 
        Returns the amount of data elements in this node's ModelFrame.
        """
        return len(self.df_node)
    
    @property
    def has_children(self):
        """ 
        Check if the node has any children.
        """
        return self.left is not None and self.right is not None
    
    @property
    def class_distribution(self):
        """
        Gives a dict containing the classes as keys and the amount of elements per class as values.
        """
        return self.df_node.target.value_counts().to_dict()
    
    @property
    def category(self):
        """
        Returns the dominant category of the elements in this node.
        """
        cd = self.class_distribution
        if len(cd) == 0:
            return 'ERROR'
        return max(cd, key=lambda key: cd[key])
    
    def add_left(self, node):
        """
        Add another node to this node as left child.
        """
        self.left = node
        
    def add_right(self, node):
        """
        Add another node to this node as right child.
        """
        self.right = node
        
    def split(self):
        """
        Split the node into two children.
        """
        if self.df_node.size <= self.min_length:
            # print('    ' * self.level, 'Leaf node #', self._id, 'based on len', self.df_node.size, '<', self.min_length)
            return
        if self.level >= self.max_depth:
            # print('    ' * self.level, 'Leaf node #', self._id, 'based on depth', self.level, '>=', self.max_depth)
            return
        if len(self.df_node['type'].unique()) <= 1:
            # print('    ' * self.level, 'Leaf node #', self._id, 'pure')
            return
        
        self.split_attr, self.split_value, l, r = find_best_split(self.df_node, self.cost_func)
        
        # print('    ' * self.level, 'Node #', self._id, 'split on', self.split_attr, self.split_value, 'L:', len(l), 'R:', len(r))
        self.left = TreeNode(self.level + 1, l, self.cost_func, self.max_depth, self.min_length).split()
        self.right = TreeNode(self.level + 1, r, self.cost_func, self.max_depth, self.min_length).split()
        return self
    
def build_tree(train_set, cost_func, max_depth, min_size):
    TreeNode.counter = 0
    root = TreeNode(0, train_set, cost_func, max_depth, min_size)
    print("Starting split at root.")
    root.split()
    print("Splitting done.")
    return root

### Print Tree

Om te kijken hoe het getrainde model eruit ziet, wordt er een (rudimentaire) functie meegegeven waarmee de boom recursief kan geprint worden. We moeten er wel voor zorgen dat de juiste velden op de juiste manier geupdated worden in de ``split`` functie van hierboven, anders krijgen we waarschijnlijk een foutboodschap.

In [23]:
def print_tree(node: TreeNode, sign='<'):
    if node.has_children:
        print(node._id, node.level * "  ", 
              node.split_attr, sign, node.split_value, '---', 
              node.cost, '---', node.class_distribution, '---', node.category)
    else:
        print(node._id, node.level * "  ", sign, "---", 
              node.cost, '---', node.class_distribution, '---', node.category)
    if node.left is not None:
        print_tree(node.left, sign="<")
    if node.right is not None:
        print_tree(node.right, sign=">=")

### Make Predictions

Uiteraard willen we het model niet alleen maar kunnen gebruiken om de trainingsdata te omschrijven (*description*), maar ook om te voorspellen hoe nieuwe data met een ongekend label geclassificeerd wordt door de boom (*prediction*). Hiervoor maken we opnieuw gebruik van recursie: we vertrekken in de *root* en vergelijken ``split_attr`` en ``split_value`` met de testdata. Op basis van deze vergelijking gaan we verder in één van de twee *children* van de *node*, totdat we een *node* bereiken die geen *children* meer heeft. De ``category`` van deze *node* is dan de uitkomst van onze voorspelling.

In [24]:
def predict(data, node: TreeNode):
    if not node.has_children:
        return node.category
    if data[node.split_attr] < node.split_value:
        return predict(data, node.left)
    return predict(data, node.right)

### Evaluate

In [25]:
def evaluate(tree, test_set):
    results = []
    for row in test_wine.itertuples():
        data = row._asdict()
        results.append((data['type'], predict(data, tree)))
    print([f"{i}: {res[0]} {res[1]}" for i, res in enumerate(results)])
    return pd.DataFrame(results, columns=['target','predicted'])

### Calculate Accuracy

In [26]:
def accuracy(tree, test_set):
    df_results = evaluate(tree, test_wine)
    return (df_results.target == df_results.predicted).sum()/len(df_results)

## Testing the Trees

### Using Entropy

In [27]:
# if 'tree_entropy' not in locals().keys():
tree_entropy = build_tree(train_wine, entropy, 8, 1)
print_tree(tree_entropy)
print("Accuracy: {:4.3f}".format(accuracy(tree_entropy, test_wine)))

Starting split at root.
 Node # 0 split on proline 660.0 L: 61 R: 63
     Node # 1 split on proline 345.0 L: 3 R: 58
         Leaf node # 2 pure
         Node # 3 split on proline 345.0 L: 0 R: 58
             Leaf node # 4 based on len 0 < 1
             Node # 5 split on proline 345.0 L: 0 R: 58
                 Leaf node # 6 based on len 0 < 1
                 Node # 7 split on proline 345.0 L: 0 R: 58
                     Leaf node # 8 based on len 0 < 1
                     Node # 9 split on proline 345.0 L: 0 R: 58
                         Leaf node # 10 based on len 0 < 1
                         Node # 11 split on proline 345.0 L: 0 R: 58
                             Leaf node # 12 based on len 0 < 1
                             Node # 13 split on proline 345.0 L: 0 R: 58
                                 Leaf node # 14 based on len 0 < 1
                                 Leaf node # 15 based on depth 8 >= 8
     Node # 16 split on proline 660.0 L: 0 R: 63
         Leaf node # 17

### Using Gini Impurity

In [29]:
# if 'tree_gini' not in locals().keys():
tree_gini = build_tree(train_wine, gini_index, 8, 1)
print_tree(tree_gini)
print("Accuracy: {:4.3f}".format(accuracy(tree_gini, test_wine)))


Starting split at root.
 Node # 0 split on proline 660.0 L: 61 R: 63
     Node # 1 split on proline 345.0 L: 3 R: 58
         Leaf node # 2 pure
         Node # 3 split on proline 345.0 L: 0 R: 58
             Leaf node # 4 based on len 0 < 1
             Node # 5 split on proline 345.0 L: 0 R: 58
                 Leaf node # 6 based on len 0 < 1
                 Node # 7 split on proline 345.0 L: 0 R: 58
                     Leaf node # 8 based on len 0 < 1
                     Node # 9 split on proline 345.0 L: 0 R: 58
                         Leaf node # 10 based on len 0 < 1
                         Node # 11 split on proline 345.0 L: 0 R: 58
                             Leaf node # 12 based on len 0 < 1
                             Node # 13 split on proline 345.0 L: 0 R: 58
                                 Leaf node # 14 based on len 0 < 1
                                 Leaf node # 15 based on depth 8 >= 8
     Node # 16 split on proline 660.0 L: 0 R: 63
         Leaf node # 17